# Explorative Data Analysis Poleno Dataset

This notebook was used to perform the Explorative Data Analysis on the Poleno dataset. It also contains code for the pre-processing and is used to create the train, validation and test split of the cleaned dataset.

In [ ]:
import os
import shutil
import sqlite3
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt

workdir = "Generative Diffusion Models for 3D Geometric Objects"
if workdir in os.getcwd() and os.path.basename(os.getcwd()) != workdir:
    os.chdir("..")

from utils import data_processing 

## Original Dataset
Download the tables from `poleno_marvel.db` and save the tables as csv

In [ ]:
db_path = "Z:/marvel/marvel-fhnw/data/Poleno/poleno_marvel.db"
csv_dir = "data/poleno/poleno_marvel/"
download_db = False

if download_db:

    # create csv folder if it doesn't exist
    Path(csv_dir).mkdir(parents=True, exist_ok=True)

    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)

    # get names of all tables
    all_tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn) # read data from database, table poleno

    # download tables
    for _, table in all_tables.iterrows():
        if not os.path.isfile(os.path.join(csv_dir, f"{table['name']}.csv")):
            print(f"Downloading table: {table['name']}")
            df = pd.read_sql_query(f"SELECT * FROM {table['name']}", conn)
            df.to_csv(os.path.join(csv_dir, f"{table['name']}.csv"), index=False)
            pass

    # close the database connection
    conn.close()

Load csv tables

In [ ]:
labels                      = pd.read_csv(os.path.join(csv_dir, "labels.csv"))
computed_data_full          = pd.read_csv(os.path.join(csv_dir, "computed_data_full.csv"))

In [ ]:
labels.info()

In [ ]:
computed_data_full.info()

The table `computed_data_full` contains a total of 968234 samples. However, there are many samples with nan values in columns that describe measured values like `bbox_area`, `convex_area` or `orientation`. 

Next are the columns that contain numerical measurements which are extractet from the shape of the corresponding sample. 

In [ ]:
sample_tags = ['event_id', 'dataset_id', 'label', 'rec_path', 'image_nr']
measurements_cols = [x for x in computed_data_full if x not in sample_tags]
measurements_cols

In [ ]:
import torch
torch.__version__

In [ ]:
print("Nr of unique Labels", len(computed_data_full["label"].value_counts()))

In [ ]:
genus = computed_data_full["label"].apply(lambda x: x.split(" ")[0])
print("Nr of different genuses:", len(set(genus)))

## Recalculation of numerical features

These measurements are extracted using `skimage.measure.regionprops()`. In order to have the maximum amount of data available for the training and to have a consistent basis for calculating these measured features, they will be recalculated from the raw images over the complete dataset.

Load `computed_data_full_re` which contains the recaltulated features.

In [ ]:
computed_data_full_re = pd.read_csv(os.path.join(csv_dir, "computed_data_full_re.csv"))

In [ ]:
computed_data_full_re.info()

In [ ]:
samples_per_event_id = computed_data_full_re["event_id"].value_counts()
valid_samples_per_event_id = samples_per_event_id[samples_per_event_id == 2].index
samples_per_event_id.value_counts()

Drop columns with more then two samples per `event_id`

In [ ]:
computed_data_full_re = computed_data_full_re.loc[computed_data_full_re["event_id"].isin(valid_samples_per_event_id)]

## Trainig Dataset

Create the dataset used for training, validation and testing.

### Maximum number of labels per dataset_id

The `label` consists of the combination of `genus` and `species/type`. As shown bellow, the dataset has only one `label` class per `dataset_id`.

In [ ]:
labels_per_dataset_id = computed_data_full_re.groupby(["dataset_id", "label"]).count().reset_index().rename(columns={"label": "num_distinct_labels"}).  \
    groupby("dataset_id").count().reset_index()[["dataset_id", "num_distinct_labels"]].sort_values("num_distinct_labels", ascending=False)
print(labels_per_dataset_id)

### Maximum number of dataset_ids with specific labels

However, a `label` might appear in multiple `dataset_id` folders. This should be taken into account when creating a train, val, test split.

In [ ]:
dataset_ids_per_label = computed_data_full.groupby(["label", "dataset_id"]).count().reset_index().rename(columns={"dataset_id": "num_dataset_folders"}). \
    groupby("label").count().reset_index()[["label", "num_dataset_folders"]].sort_values("num_dataset_folders", ascending=False)
print(dataset_ids_per_label)

Drop columns with any missing values

In [ ]:
computed_data_full_filtered = computed_data_full_re.dropna()
print(f"Samples in original dataset: {len(computed_data_full)}\nAfter dropping samples with nan : {len(computed_data_full_filtered)}")

remove labels with small sample count

In [ ]:
min_samples_per_label = 2000

samples_per_label = computed_data_full_filtered.value_counts("label").reset_index()
plt.figure(figsize=(14, 5))
plt.bar(x=samples_per_label["label"], height=samples_per_label["count"])
plt.axhline(min_samples_per_label, color='red', ls='dotted')
plt.xticks(rotation=90)
plt.title("Label distribution in the dataset")
plt.show()

The visualization above shows the number of samples per label class. A class is made up of the genus and the species. There are several species per genus. Since some species are indicated with placeholders, e.g. `sp. 0` or `sp. 1`, it is not possible to know exactly how they relate to each other.  

In a next step the labels with a sample count below `min_samples_per_label` are dropped

In [ ]:
valid_labels = list(samples_per_label.loc[samples_per_label["count"] >= min_samples_per_label]["label"])
computed_data_full_filtered = computed_data_full_filtered.loc[computed_data_full_filtered["label"].isin(valid_labels)]

Next the feature columns are normalized per column

In [ ]:
def normalize(df):
    return (df-df.mean())/df.std()

computed_data_full_filtered[measurements_cols] = normalize(computed_data_full_filtered[measurements_cols])

Furthermore a `filenames` column is created for easyer sampling during training

In [ ]:
computed_data_full_filtered["filenames"] = computed_data_full_filtered["dataset_id"] + "/" + computed_data_full_filtered["rec_path"] 
computed_data_full_filtered["filenames"].iloc[0]

For the same reason a `class_id` column is created

In [ ]:
class_to_id = {c:i for i, c in enumerate(set(list(computed_data_full_filtered["dataset_id"])))}
computed_data_full_filtered["class_id"] = computed_data_full_filtered["dataset_id"].apply(lambda x: class_to_id[x])

label_to_id = {c:i for i, c in enumerate(set(list(computed_data_full_filtered["label"])))}
computed_data_full_filtered["label_id"] = computed_data_full_filtered["label"].apply(lambda x: label_to_id[x])

To review the zero shot capabilities of the model, it is necessary that the test data set contains new species unknown to the model. Due to the uncertainties mentioned above, the **unknown classes are selected based on genus**.

In [ ]:
genus = computed_data_full_filtered["label"].apply(lambda x: x.split(" ")[0])
print("Nr of different genuses:", len(set(genus)))

In [ ]:
print("Nr of unique Labels", len(computed_data_full_filtered["label"].value_counts()))

In [ ]:
# samples_per_label.loc[:, "genus"] = samples_per_label["label"].apply(lambda x: x.split(" ")[0])
# # samples_per_label = samples_per_label.sort_values("genus")

# plt.figure(figsize=(14, 5))
# plt.bar(x=samples_per_label["genus"], height=samples_per_label["count"])
# plt.axhline(min_samples_per_label, color='red', ls='dotted')
# plt.xticks(rotation=90)
# plt.title("Label distribution in the dataset")
# plt.show()

In [ ]:
# samples_per_genus = samples_per_label[["genus", "count"]].groupby("genus").sum().sort_values(by="count").reset_index()
# # samples_per_genus
# list(samples_per_genus.head(5)["genus"])

The labels from the genuses `Lolium`, `Phytophthora`, `Arrhenatherum`, `Holcus` and `Cynosurus` are selected to only appear in the test set.

In [ ]:
# eval_only = ['Lolium', 'Phytophthora', 'Arrhenatherum', 'Holcus', 'Cynosurus']

# # select columns with test only genuses
# eval_only = "|".join(test_genus for test_genus in eval_only)
# df_eval_only = computed_data_full_filtered.loc[computed_data_full_filtered["label"].str.contains(eval_only)]

# # remove them from computed_data_full_filtered
# computed_data_full_filtered = computed_data_full_filtered.drop(index=df_eval_only.index)

The dataset is subsequently split into a train validation test dataset. The spliting balances the amount of labels in the validation and the test set. For the test set, an amount of 25 events (50 samples) per label is selected. For the validation set, an amount of 10 (20 samples) events is selected per dataset. The other samples are present in the test set.

In [ ]:
def sample_events(df, n, random_state=42):

    """
    This function samples events from a dataframe based on their labels.

    Parameters:
    df (pandas.DataFrame): The input dataframe. It should contain columns 'event_id' and 'label'.
    n (int): The number of events to sample from each label.
    random_state (int, optional): The seed for the random number generator. Default is 42.

    Returns:
    pandas.DataFrame: A dataframe containing the sampled events.

    The function first identifies unique events in the input dataframe. It then samples 'n' events from each label 
    using the 'sample' method of pandas.DataFrame.groupby. The sampled events are returned as a dataframe.
    """

    events_in_classes = df[["event_id", "label"]].drop_duplicates()
    sampled_events = events_in_classes.groupby("label").sample(n, random_state=random_state)
    sampled_event_ids = list(sampled_events["event_id"])

    return df[df["event_id"].isin(sampled_event_ids)]

    

test = sample_events(computed_data_full_re, n=3)

In [ ]:
random_state = 42
n_samples_test = 25
n_samples_val = 25
recreate_datasets = True

if recreate_datasets:

    dataset_dir =  "."
    # dataset_dir = "Z:\simon_luder\Generative_Diffusion_Models_for_3D_Geometric_Objects"

    Path(dataset_dir).mkdir(parents=True, exist_ok=True)

    # #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # computed_data_full_filtered = computed_data_full_filtered.loc[computed_data_full_filtered["dataset_id"].isin(["11ee037f-6ea4-cbb8-a9fd-66f2ec8a65cb"])]

    # set df_train as the complete computed_data_full_filtered
    df_train = computed_data_full_filtered

    # sample 
    # df_test = df_train.groupby("label").sample(n_samples_test, random_state=random_state)
    df_test = sample_events(df_train, n=n_samples_test, random_state=random_state)
    # remove samples from df_test in df_train
    df_train = df_train.drop(index=df_test.index)

    # # append evaluation only genuses
    # # df_test_eval_only = df_eval_only.groupby("label").sample(n_samples_test, random_state=random_state)
    # df_test_eval_only = sample_events(df_eval_only, n=n_samples_test, random_state=random_state)
    # df_test = pd.concat([df_test, df_test_eval_only])

    # df_val = df_train.groupby("label").sample(n_samples_val, random_state=random_state)
    df_val = sample_events(df_train, n=n_samples_val, random_state=random_state)
    # remove samples from df_val in df_train
    df_train = df_train.drop(df_val.index)

    df_train.to_csv(os.path.join(dataset_dir, "labels_train.csv"), index=False)
    df_val.to_csv(os.path.join(dataset_dir, "labels_val.csv"), index=False)
    df_test.to_csv(os.path.join(dataset_dir, "labels_test.csv"), index=False)

    print(f"Generated Datasets \nTrain: {len(df_train)} \nVal: {len(df_val)} ({n_samples_val} per class) \nTest: {len(df_test)} ({n_samples_test} per class)")

### Sandbox

The code which follofs from here on has been used only for debugging

In [ ]:
train_old = pd.read_csv("Z:\simon_luder\Generative_Diffusion_Models_for_3D_Geometric_Objects/labels_train.csv")
val_old = pd.read_csv("Z:\simon_luder\Generative_Diffusion_Models_for_3D_Geometric_Objects/labels_val.csv")
test_old = pd.read_csv("Z:\simon_luder\Generative_Diffusion_Models_for_3D_Geometric_Objects/labels_test.csv")

label_to_id = {c:i for i, c in enumerate(set(list(train_old["label"])))}

train_old["label_id"] = train_old["label"].apply(lambda x: label_to_id[x])
val_old["label_id"] = val_old["label"].apply(lambda x: label_to_id[x])
test_old["label_id"] = test_old["label"].apply(lambda x: label_to_id[x])

train_old.to_csv("labels_train.csv", index=False, encoding="utf-8")
val_old.to_csv("labels_val.csv", index=False, encoding="utf-8")
test_old.to_csv("labels_test.csv", index=False, encoding="utf-8")

# update with genus id

In [ ]:
train_old = pd.read_csv("Z:\simon_luder\Generative_Diffusion_Models_for_3D_Geometric_Objects/labels_train.csv")
val_old = pd.read_csv("Z:\simon_luder\Generative_Diffusion_Models_for_3D_Geometric_Objects/labels_val.csv")
test_old = pd.read_csv("Z:\simon_luder\Generative_Diffusion_Models_for_3D_Geometric_Objects/labels_test.csv")

genus_to_id = {c:i for i, c in enumerate(set([x[0] for x in train_old["label"].str.split()]))}

train_old["genus_id"] = train_old["label"].apply(lambda x:genus_to_id[x.split()[0]])
val_old["genus_id"] = val_old["label"].apply(lambda x:genus_to_id[x.split()[0]])
test_old["genus_id"] = test_old["label"].apply(lambda x:genus_to_id[x.split()[0]])

train_old.to_csv("labels_train.csv", index=False, encoding="utf-8")
val_old.to_csv("labels_val.csv", index=False, encoding="utf-8")
test_old.to_csv("labels_test.csv", index=False, encoding="utf-8")

In [ ]:
model_names = [
    "ldm_cls_8_512_e5",
    # "ldm_clstbl_8_512_e5x",
    "ldm_clstbl_8_512_e5",
    "ldm_tbl_8_512_e5x",
    # "ldm_tbl_8_512_e5",
]

In [ ]:
for model_name in model_names:
    testset = pd.read_csv("Z:\simon_luder\Generative_Diffusion_Models_for_3D_Geometric_Objects/labels_test.csv")
    testset["filenames"] = testset["filenames"].apply(lambda x: os.path.join(f"holographic_pollen/{model_name}/test/images/", x.split("/")[-1]))
    testset.to_csv(f"labels_test_{model_name}.csv", index=False, encoding="utf-8")

In [ ]:
label_to_id = {c:i for i, c in enumerate(set(list(train_old["label"])))}
train_old["label_id"] = train_old["label"].apply(lambda x: label_to_id[x])

In [ ]:
train_old = pd.read_csv("Z:\simon_luder\Generative_Diffusion_Models_for_3D_Geometric_Objects/labels_train.csv")
train_new = pd.read_csv("labels_train.csv")

In [ ]:
import pandas as pd
train_new = pd.read_csv("labels_train.csv")
test = train_new.groupby("genus_id").sample(1).reset_index(drop=True)

In [ ]:
test.to_csv("labels_train_mini.csv")

In [ ]:
print(df_train["label"].value_counts().to_latex())

Creation of a debug dataset that only contains one sample per `class_id`

In [ ]:
random_state = 42
dataset_dir = "Z:\simon_luder\Generative_Diffusion_Models_for_3D_Geometric_Objects"
create_debug_set = False

if create_debug_set:

    df = pd.read_csv(os.path.join(dataset_dir, "labels_test.csv"))
    df = df.groupby("class_id").sample(1, random_state=random_state)
    df.to_csv(os.path.join(dataset_dir, "labels_test_mini.csv", index = False))

    df = pd.read_csv(os.path.join(dataset_dir, "labels_val.csv"))
    df = df.groupby("class_id").sample(1, random_state=random_state)
    df.to_csv(os.path.join(dataset_dir, "labels_val_mini.csv", index = False))

    df = pd.read_csv(os.path.join(dataset_dir, "labels_train.csv"))
    df = df.groupby("class_id").sample(1, random_state=random_state)
    df.to_csv(os.path.join(dataset_dir, "labels_train_mini.csv", index = False))

In [ ]:
import pandas as pd

df1=pd.read_csv("Z:\simon_luder\Generative_Diffusion_Models_for_3D_Geometric_Objects/labels_val.csv")
df2=pd.read_csv("Z:\simon_luder\Generative_Diffusion_Models_for_3D_Geometric_Objects/labels_train.csv")

# Assuming df1 and df2 are your DataFrames
overlap = pd.merge(df1, df2, how='inner')

# Check if the overlap DataFrame is empty
if not overlap.empty:
    print("There are overlapping rows.")
else:
    print("No overlapping rows.")